In [1]:
import pandas
from wandb import magic
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


wandb: Wandb version 0.8.22 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np

# predicts 7/12, 8/11 or 6/8
sequences = ['H375S', 'WT', 'LM', 'LM_H375S', 'H375S_H61Y', 'H375S_Q105H', 'H375S_V108I', 'H375S_Q105H_V108I', 'H375S_NIK474_476DMR', 'H375S_Q105H_V108I_NIK474_476DMR', 'H375T', 'LM_H375T']
#sequences = ['H375S', 'WT', 'LM', 'LM_H375S', 'H375S_H61Y', 'H375S_Q105H', 'H375S_V108I', 'H375S_Q105H_V108I', 'H375S_Q105H_V108I_NIK474_476DMR', 'H375T', 'LM_H375T']
#sequences = ['H375S', 'H375S_H61Y', 'H375S_Q105H', 'H375S_V108I', 'H375S_Q105H_V108I', 'H375S_NIK474_476DMR', 'H375S_Q105H_V108I_NIK474_476DMR', 'H375T']
#sequences = ['H375S']

active = [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]  
#active = [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1] 
#active = [0, 0, 0, 0, 0, 1, 0, 1] 
#active = [0]


my_dict = {}

for seq,act in zip(sequences,active):

    df=pd.read_csv('output-rgyr-%s.dat'%seq, sep=" ",header=None)
    df1=pd.read_csv('output-charge-%s.dat'%seq, sep=" ",header=None)
    df2=pd.read_csv('output-dot-%s.dat'%seq, sep=" ",header=None)
    df3=pd.read_csv('output-%s.dat'%seq, sep=" ",header=None)
    df4=pd.read_csv('output-pt0-mag-%s.dat'%seq, sep=" ",header=None)
    df5=pd.read_csv('output-charge-std-%s.dat'%seq, sep=" ",header=None)
    my_dict[seq]=[df4,act]
    #input data is df4, which is the average dipole moment per time step across the 5 replicate trajectories
    #y = np.array(my_dict['H375S'])
    
    

In [3]:
a = []
#Here I am adding each first column of the data as an array per sequence
#For example, the average dipole moment across 5 replicas of the first timestep for H375S is: -3.67851 -2.19384 1.71885
for seq in sequences:
    a.append(my_dict[seq][0].values)
#print(a)

In [4]:
#a = a.reshape(a.shape[0], -1)  
a = np.asarray(a)
a = a.reshape(a.shape[0], -1)
print (a)
X = a
Y = active
#print (b)

[[ 3.63005  6.52154  5.75852  5.12014  5.44543]
 [ 6.27117  5.75019  7.8948   4.84391  3.24225]
 [ 6.63515  6.45865  4.68326  8.20309  5.68154]
 [10.8525   7.23245 10.6234   5.11686  4.84477]
 [ 6.90239  9.81281 10.731   12.7027   7.0801 ]
 [ 5.32339 10.7344   6.79031 10.5846   9.84809]
 [11.0906   6.16148  5.02688  5.85155  3.7702 ]
 [ 3.97945 12.8192   6.14302  7.61566  8.26343]
 [ 6.58074  8.26008  2.76228  4.15795  2.73069]
 [ 6.51494 16.4857   8.19625  1.80879  4.09538]
 [ 3.59118  6.39338  3.03531  3.24092  4.43871]
 [ 6.39996  6.06392  1.97374  3.88145  6.14879]]


In [5]:
#the reason the shape of a below is 12, 60000 is that there are 12 sequences and 20000 timesteps x 3 coordinates = 60000 features
a.shape

(12, 5)

In [6]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [7]:
model = Sequential()
model.add(Dense(5, input_dim=5, activation='relu'))
model.add(Dense(4, activation='relu')) 
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X, Y, epochs=300, batch_size=100, verbose=0)
# make class predictions with the model
predictions = model.predict_classes(X)

wandb: Wandb version 0.8.22 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [8]:
sequences2 = ['H375S_H61Y_Q105H_V108I', 'H375T_H61Y_Q105H_V108I', 'H61Y_Q105H_V108I']
#sequences = ['H375S']
my_dict2 = {}

for seq2 in sequences2:

    df=pd.read_csv('output-rgyr-%s.dat'%seq2, sep=" ",header=None)
    df1=pd.read_csv('output-charge-%s.dat'%seq2, sep=" ",header=None)
    df2=pd.read_csv('output-dot-%s.dat'%seq, sep=" ",header=None)
    df3=pd.read_csv('output-%s.dat'%seq, sep=" ",header=None)
    df4=pd.read_csv('output-pt0-mag-%s.dat'%seq, sep=" ", header=None)
    df5=pd.read_csv('output-charge-std-%s.dat'%seq, sep=" ",header=None)
    #df=0 
    my_dict2[seq2]=[df4]
   
#df.columns= my_dict["H375S"][0].columns
#df1.columns=my_dict["H375S"][0].columns
#distance2 = my_dict2["H375S_H61Y_Q105H_V108I"][0].columns

In [9]:
c = []
for seq2 in sequences2:
    c.append(my_dict2[seq2][0].values)
    #c = c.tolist()
#print(c)
c = np.asarray(c)

c = c.reshape(c.shape[0], -1)
#c.shape

In [10]:
reference = np.array(active)
t = model.predict_classes(X)
for i in range(12):
    print('(predicted %d)' % (t[i]))
for x in range(12):
    print (x)
    z = np.delete(a, x, 0)
    y = np.delete(active, x, 0)
    model.fit(z, y, epochs=150, batch_size=10, verbose=0)
    #print (y)
    t = model.predict_classes(X)
    #for i in range(12):
        #print('%d (expected %d)' % (t[i], Y[i]))
    #t=t.reshape(-1, 1).tolist()
    zz=[tt[0] for tt in t]
    #print (zz)
    #print (Y)
    e = zz == Y
    #print (e)
    if np.all(e):
        print("yes")
    else:   
        print ("no")

wandb: Wandb version 0.8.22 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


(predicted 0)
(predicted 1)
(predicted 0)
(predicted 1)
(predicted 0)
(predicted 0)
(predicted 1)
(predicted 0)
(predicted 1)
(predicted 1)
(predicted 0)
(predicted 1)
0


/home/natasha/.local/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.422567). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)
/home/natasha/.local/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.215000). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


no
1
no
2
no
3
no
4
no
5
no
6
no
7
yes
8
yes
9
yes
10
yes
11
yes


In [11]:
predictions = model.predict_classes(a)
for i in range(12):
    print('%d (expected %d)' % (predictions[i], Y[i]))

0 (expected 0)
0 (expected 0)
0 (expected 0)
1 (expected 1)
0 (expected 0)
0 (expected 0)
0 (expected 0)
0 (expected 0)
1 (expected 1)
0 (expected 0)
1 (expected 1)
1 (expected 1)


In [12]:
predictions = model.predict_classes(c)
for i in range(3):
    print('%d' % (predictions[i]))

1
1
1
